<a href="https://colab.research.google.com/github/habilg/keras_advanced/blob/master/keras_for_researchers_03_CustomLoss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras


In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
housing = fetch_california_housing()

In [5]:
x_train, x_test, y_train, y_test = train_test_split(housing["data"], housing["target"])

In [6]:
sc=StandardScaler()
x_train = sc.fit_transform(x_train)
x_test= sc.fit_transform(x_test)

In [27]:
x_test[0]

array([-1.41837093, -0.14493256, -0.38218457, -0.06313802, -0.53786064,
       -0.58085813,  0.33664405,  0.13665653])

In [33]:
print(housing.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

In [7]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(50, activation='relu'))
model.add(keras.layers.Dense(10,activation='relu'))
model.add(keras.layers.Dense(1))

In [42]:
model.compile(loss="mse",
              optimizer="sgd",
              metrics=["mean_absolute_error"])

In [43]:
model.fit(x_train, y_train,epochs=20, validation_split=0.15)

Epoch 1/20
412/412 [==============================] - 1s 1ms/step - loss: 0.8022 - mean_absolute_error: 0.6078 - val_loss: 0.4799 - val_mean_absolute_error: 0.4888
Epoch 2/20
412/412 [==============================] - 1s 1ms/step - loss: 0.4253 - mean_absolute_error: 0.4689 - val_loss: 0.4125 - val_mean_absolute_error: 0.4513
Epoch 3/20
412/412 [==============================] - 0s 1ms/step - loss: 0.4176 - mean_absolute_error: 0.4619 - val_loss: 0.4127 - val_mean_absolute_error: 0.4696
Epoch 4/20
412/412 [==============================] - 0s 1ms/step - loss: 0.3802 - mean_absolute_error: 0.4430 - val_loss: 0.3861 - val_mean_absolute_error: 0.4408
Epoch 5/20
412/412 [==============================] - 0s 1ms/step - loss: 0.3698 - mean_absolute_error: 0.4348 - val_loss: 0.3636 - val_mean_absolute_error: 0.4333
Epoch 6/20
412/412 [==============================] - 0s 1ms/step - loss: 0.3636 - mean_absolute_error: 0.4316 - val_loss: 0.3639 - val_mean_absolute_error: 0.4264
Epoch 7/20
412/4

## Step 1 - new loss definition

In [44]:
def my_loss(y_true, y_predict):
  error = tf.abs(y_true - y_predict)
  return tf.experimental.numpy.select([error<0.1 , error<0.5, error>=0.5],[error, error*2, error**2])

In [ ]:
model.compile(loss=my_loss,
              optimizer="sgd",
              metrics=["mean_absolute_error"])

## Step 2- provide parameters for loss function

In [45]:
def loss_wrapper(t1,t2):
  def my_loss(y_true, y_predict):
    error = tf.abs(y_true - y_predict)
    return tf.experimental.numpy.select([error<t1 , error<t2, error>=t2],[error, error*2, error**2])
  return my_loss

In [46]:
model.compile(loss=loss_wrapper(0.1,0.5),
              optimizer="sgd",
              metrics=["mean_absolute_error"])

In [47]:
model.fit(x_train, y_train,epochs=5, validation_split=0.15)

Epoch 1/5
412/412 [==============================] - 1s 1ms/step - loss: 0.5880 - mean_absolute_error: 0.4007 - val_loss: 0.5927 - val_mean_absolute_error: 0.4005
Epoch 2/5
412/412 [==============================] - 1s 1ms/step - loss: 0.5809 - mean_absolute_error: 0.3972 - val_loss: 0.5980 - val_mean_absolute_error: 0.3973
Epoch 3/5
412/412 [==============================] - 0s 1ms/step - loss: 0.5801 - mean_absolute_error: 0.3951 - val_loss: 0.6208 - val_mean_absolute_error: 0.4109
Epoch 4/5
412/412 [==============================] - 0s 1ms/step - loss: 0.5751 - mean_absolute_error: 0.3925 - val_loss: 0.5933 - val_mean_absolute_error: 0.4054
Epoch 5/5
412/412 [==============================] - 1s 1ms/step - loss: 0.5737 - mean_absolute_error: 0.3904 - val_loss: 0.6153 - val_mean_absolute_error: 0.4148


if the model is saved, the parameters are not passed. you should provide the model with these parameters while you are loading it. let's give it a try

In [48]:
model.save("mymodel.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [49]:
loaded_model=keras.models.load_model("mymodel.h5")

ValueError: Unknown loss function: 'my_loss'. Please ensure you are using a `keras.utils.custom_object_scope` and that this object is included in the scope. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

to avoid the last error, the loss should be passed as a ``custom object `` argument like below. Remeber that the provided key must be the same name of the written function.

In [52]:
loaded_model=keras.models.load_model("mymodel.h5", custom_objects={'my_loss':loss_wrapper(0.1,0.5)})

## Step 3 -subclassing

```python  
class Myloss(tf.keras.losses.Loss):
  def __init__(self,...,**kwargs):
    # initilizing the parameters
    super().__init__(**kwargs)

  def call(self, y_true,y_predict) / or __call__():
    
    #body of calculations

  
  def get_config(self):
    parent_config= super().get_config()
    return {**parent_config, "key1":value, ..., "keyn":value}



```

In [17]:
class Myloss(tf.keras.losses.Loss):
  def __init__(self,t1,t2,**kwargs):
    super().__init__(**kwargs)
    self.t1 = t1
    self.t2 = t2


  def call(self, y_true,y_predict):

    error = tf.abs(y_true - y_predict)
    return tf.experimental.numpy.select([error<self.t1 , error<self.t2, error>=self.t2],[error, error*2, error**2])


  def get_config(self):
    parent_config= super().get_config()
    return {**parent_config, "t1":self.t1, "t2":self.t2}


In [18]:
model.compile(loss=Myloss(0.1,0.5),
              optimizer="sgd",
              metrics=["mean_absolute_error"])

In [19]:
model.fit(x_train, y_train,epochs=5, validation_split=0.15)

Epoch 1/5
412/412 [==============================] - 1s 1ms/step - loss: 0.6301 - mean_absolute_error: 0.4328 - val_loss: 0.6484 - val_mean_absolute_error: 0.4328
Epoch 2/5
412/412 [==============================] - 1s 1ms/step - loss: 0.6232 - mean_absolute_error: 0.4279 - val_loss: 0.6373 - val_mean_absolute_error: 0.4397
Epoch 3/5
412/412 [==============================] - 1s 1ms/step - loss: 0.6173 - mean_absolute_error: 0.4240 - val_loss: 0.6644 - val_mean_absolute_error: 0.4385
Epoch 4/5
412/412 [==============================] - 1s 1ms/step - loss: 0.6058 - mean_absolute_error: 0.4182 - val_loss: 0.6157 - val_mean_absolute_error: 0.4189
Epoch 5/5
412/412 [==============================] - 1s 1ms/step - loss: 0.5960 - mean_absolute_error: 0.4128 - val_loss: 0.7208 - val_mean_absolute_error: 0.4853


In [20]:
model.save("mymodel+subclassed_loss.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [21]:
loaded_model = keras.models.load_model("mymodel+subclassed_loss.h5",custom_objects={'Myloss':Myloss})